In [43]:
import pandas as pd

data_path = './data/car-sim1/'
img_path = './data/car-sim1/IMG/'
df = pd.read_csv(data_path+'driving_log.csv', header=None)

df.head()

,0,1,2,3,4,5,6
0,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000006
1,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000012
2,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000006
3,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000004
4,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.0,0,0.000010


In [44]:
import csv
import cv2
import numpy as np

lines = []
with open(data_path+'driving_log.csv') as csvFile:
    reader = csv.reader(csvFile)
    #next(reader, None)
    for line in reader:
        lines.append(line)
        
def process_image(img): 
    return img

        
images = []
measurements = []
for line in lines:
    '''
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = img_path + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    '''
    
    steering_center = float(line[3])

    # create adjusted steering measurements for the side camera images
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    steering_right = steering_center - correction

    # read in images from center, left and right cameras
    filename = line[0].split('/')[-1]
    img_center = process_image(cv2.imread(img_path + filename))
    filename = line[1].split('/')[-1]
    img_left = process_image(cv2.imread(img_path + filename))
    filename = line[2].split('/')[-1]
    img_right = process_image(cv2.imread(img_path + filename))

    # add images and angles to data set
    images.extend([img_center, img_left, img_right])
    measurements.extend([steering_center, steering_left, steering_right])
    
X_train = np.array(images)
Y_train = np.array(measurements)

print('X_train:', X_train.shape)
print('Y_train:', Y_train.shape)

X_train: (6861, 160, 320, 3)
Y_train: (6861,)


In [45]:
augmented_images = []
augmented_measurements = []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
X_train = np.array(augmented_images)
Y_train = np.array(augmented_measurements)

print('X_train:', X_train.shape)
print('Y_train:', Y_train.shape)

X_train: (13722, 160, 320, 3)
Y_train: (13722,)


## First Model

In [33]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.optimizers import Adam

model = Sequential()
#model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(lr=0.0001))
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, epochs=2)

model.save('model.h5')

Train on 10972 samples, validate on 2744 samples
Epoch 1/2
10972/10972 [==============================] - 7s 677us/step - loss: 0.0980 - val_loss: 0.1220
Epoch 2/2
10972/10972 [==============================] - 7s 623us/step - loss: 0.0578 - val_loss: 0.1509


## Second Model

In [46]:
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Flatten, Dense, Lambda, Cropping2D

model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5))
model.add(Conv2D(6, (5,5), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(6, (5,5), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(lr=0.001))
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model.h5')

NameError: name 'Cropping2D' is not defined

In [10]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1888059787822219105
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 302448640
locality {
  bus_id: 1
}
incarnation: 3621040110653449976
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:09:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 338100224
locality {
  bus_id: 1
}
incarnation: 4712016052853008706
physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1"
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 338100224
locality {
  bus_id: 1
}
incarnation: 4475804696287490450
physical_device_desc: "device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:42:00.0, compute capability: 6.1"
, name: "/device:GPU:3"
device_type: "GPU"
memory_limit: 338100224
locality {
  bus_id: 1
}
incarnation: 10865371839291708873
physical_device_desc: "device: 3, name: GeForce GTX